<a href="https://colab.research.google.com/github/WMaia9/test/blob/main/Mori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from pathlib import Path
import re
import pandas as pd

In [9]:
# Carregar novamente o conteúdo do arquivo corrigido
file_path = Path("/content/1_Devolutiva_dim_pedag_planejamento.md")
with open(file_path, encoding="utf-8") as f:
    texto = f.read()

In [10]:
# Função para extrair as seções por rubrica e nível
def extrair_devolutivas(texto):
    # Regex mais tolerante a espaços, quebras de linha e variações
    pattern = r'### Rubrica (\d+) – ([^-]+) – Nível ([^\n]+)\s+\*\*Seus pontos fortes:\*\*\s*(.*?)\s+\*\*O que fazer para avançar\?\*\*\s*(.*?)\s+\*\*Necessidades formativas\*\*\s*(.*?)(?=\n### Rubrica|\Z)'
    matches = re.findall(pattern, texto, re.DOTALL)

    dados = []
    for rubrica_num, titulo, nivel, pontos_fortes, avancar, formativas in matches:
        dados.append({
            'rubrica': f'Rubrica {rubrica_num} - {titulo.strip()}',
            'nivel': nivel.strip(),
            'pontos_fortes': pontos_fortes.strip(),
            'avancar': avancar.strip(),
            'formativas': formativas.strip()
        })
    return pd.DataFrame(dados)

In [11]:
# Separar os blocos com base no padrão de rubrica
blocos = re.split(r'(?=^### Rubrica \d+ – .+? – Nível .+$)', texto, flags=re.MULTILINE)

dados = []

for bloco in blocos:
    bloco = bloco.strip()
    if not bloco.startswith("### Rubrica"):
        continue

    cabecalho = re.match(r'^### Rubrica (\d+) – (.+?) – Nível (.+)$', bloco, flags=re.MULTILINE)
    if not cabecalho:
        continue

    rubrica_num = cabecalho.group(1).strip()
    rubrica_nome = cabecalho.group(2).strip()
    nivel = cabecalho.group(3).strip()

    conteudo = bloco[cabecalho.end():].strip()

    secao_regex = re.compile(
        r'\*\*Seus pontos fortes\*\*[:]*\s*(.*?)\s*'
        r'\*\*O que fazer para avançar\?\*\*\s*(.*?)\s*'
        r'\*\*Necessidades formativas\*\*\s*(.*)',
        flags=re.DOTALL
    )

    secoes = secao_regex.search(conteudo)
    if not secoes:
        continue

    pontos_fortes = secoes.group(1).strip()
    avancar = secoes.group(2).strip()
    formativas = secoes.group(3).strip()

    dados.append({
        "rubrica": f"Rubrica {rubrica_num} - {rubrica_nome}",
        "nivel": nivel,
        "pontos_fortes": pontos_fortes,
        "avancar": avancar,
        "formativas": formativas
    })

# Gerar DataFrame e exibir
df_devolutivas = pd.DataFrame(dados)

In [12]:
df_devolutivas

,rubrica,nivel,pontos_fortes,avancar,formativas
0,Rubrica 1 - Sensibilização,Avançar,Muito bem! Você já reconhece a importância do ...,"Para avançar, será necessário estar preparado(...",1. **Diagnóstico escolar para embasar o plane...
1,Rubrica 2 - Exploração,Consolidar,Ótimo! Você já se sente preparado(a) para assu...,"Antes de avançar, será necessário fortalecer s...",1. **Diagnóstico escolar para embasar o plane...
2,Rubrica 2 - Exploração,Avançar,Parabéns! Você já se sente preparado(a) para d...,"Para avançar, será fundamental estar preparado...",1. **Diagnóstico escolar para embasar o plane...
3,Rubrica 3 - Liderança Estratégica,Avançar,Excelente trabalho! Você já se sente preparado...,"Para avançar, será necessário fortalecer o pre...",1. **Interdisciplinariedade e práticas integr...
4,Rubrica 4 - Transformação cultural,Consolidar,Fantástico! Você já se sente preparado(a) para...,Embora este nível reflita uma atuação exemplar...,1. **Interdisciplinariedade e práticas integr...


In [13]:
# Função para mapear a pontuação para rubrica e nível

def pontuacao_para_rubrica_nivel(pontuacao: int):
    if 0 <= pontuacao <= 5:
        return "Rubrica 1 - Sensibilização", "Consolidar"
    elif 6 <= pontuacao <= 9:
        return "Rubrica 1 - Sensibilização", "Avançar"
    elif 10 <= pontuacao <= 16:
        return "Rubrica 2 - Exploração", "Consolidar"
    elif 17 <= pontuacao <= 21:
        return "Rubrica 2 - Exploração", "Avançar"
    elif 22 <= pontuacao <= 30:
        return "Rubrica 3 - Liderança Estratégica", "Consolidar"
    elif 31 <= pontuacao <= 36:
        return "Rubrica 3 - Liderança Estratégica", "Avançar"
    elif 37 <= pontuacao <= 51:
        return "Rubrica 4 - Transformação cultural", "Consolidar"
    else:
        return None, None  # Fora da faixa prevista

# Exemplo de uso: pontuação = 17 (esperado: Rubrica 2 - Exploração, Avançar)
rubrica, nivel = pontuacao_para_rubrica_nivel(17)

rubrica, nivel

('Rubrica 2 - Exploração', 'Avançar')

In [14]:
# Buscar a devolutiva correspondente à rubrica e nível obtidos
devolutiva = df_devolutivas[
    (df_devolutivas['rubrica'] == rubrica) &
    (df_devolutivas['nivel'] == nivel)
]

# Exibir o conteúdo encontrado
if not devolutiva.empty:
    resultado = devolutiva.iloc[0]
    {
        "rubrica": resultado["rubrica"],
        "nivel": resultado["nivel"],
        "pontos_fortes": resultado["pontos_fortes"][:300] + "...",
        "avancar": resultado["avancar"][:300] + "...",
        "formativas": resultado["formativas"][:300] + "..."
    }
else:
    "Devolutiva não encontrada para essa pontuação."

In [16]:
# Função completa para gerar devolutiva textual a partir da pontuação
def gerar_devolutiva(pontuacao: int, df: pd.DataFrame) -> str:
    rubrica, nivel = pontuacao_para_rubrica_nivel(pontuacao)
    if not rubrica:
        return f"❌ Pontuação {pontuacao} está fora da faixa válida (0 a 51)."

    resultado = df[
        (df["rubrica"] == rubrica) &
        (df["nivel"] == nivel)
    ]

    if resultado.empty:
        return f"❌ Não foi possível encontrar a devolutiva para {rubrica} - {nivel}."

    item = resultado.iloc[0]
    texto = f"""
🔢 **Pontuação:** {pontuacao}
🏷️ **Rubrica:** {item['rubrica']}
📊 **Nível:** {item['nivel']}

---

✅ **Seus pontos fortes:**

{item['pontos_fortes']}

---

📈 **O que fazer para avançar:**

{item['avancar']}

---

📚 **Necessidades formativas:**

{item['formativas']}
"""
    return texto.strip()


In [21]:
from IPython.display import Markdown
exemplo_pontuacao = 17
texto_md = gerar_devolutiva(exemplo_pontuacao, df_devolutivas)
Markdown(texto_md)

🔢 **Pontuação:** 17
🏷️ **Rubrica:** Rubrica 2 - Exploração
📊 **Nível:** Avançar

---

✅ **Seus pontos fortes:**

Parabéns! Você já se sente preparado(a) para desempenhar um papel ativo na mediação do planejamento pedagógico, garantindo que ele seja baseado em um diagnóstico de aprendizagem dos estudantes. Você se percebe preparado(a) para promover discussões importantes sobre o alinhamento do planejamento com as políticas educacionais e a proposta pedagógica da escola, além de monitorar se as ações estão sendo implementadas conforme planejado.

---

📈 **O que fazer para avançar:**

Para avançar, será fundamental estar preparado(a) para ampliar o escopo do diagnóstico, estruturar melhor as rotinas de monitoramento e replanejamento e fornecer apoio consistente aos professores, com diretrizes claras sobre o que deve conter no planejamento e devolutivas construtivas. Isso inclui estar preparado para continuamente avaliar e revisar as estratégias de recomposição das aprendizagens. Veja a seguir algumas indicações de objetos educacionais e recursos de aprendizagem que podem apoiá-lo nessa tarefa.

---

📚 **Necessidades formativas:**

1.  **Diagnóstico escolar para embasar o planejamento pedagógico**
    *   Diagnóstico das práticas de ensino dos professores (sua capacidade de transpor os parâmetros curriculares em práticas efetivas de ensino e aprendizagem)
2.  **Planejamento das ações, responsáveis e recursos necessários**
    *   Conhecimento de estratégias didático-metodológicas para aprendizagem mais efetiva (metodologias ativas, aprendizagem por projetos, práticas de diferenciação e personalização do ensino)
    *   Elaboração de diretrizes para o planejamento macro das ações e para o seu desdobramento progressivo em planos de aula detalhados
    *   Conhecimento de estratégias específicas de recomposição das aprendizagens e diretrizes para sua integração ao planejamento pedagógico.
3.  **Suporte e devolutiva aos professores**
    *   Avaliação e devolutiva técnico/crítica do planejamento pedagógico dos professores (critérios de qualidade, roteiros de análise e devolutiva, técnicas de feedback do gestor e entre pares).
4.  **Monitoramento, avaliação e replanejamento**
    *   Monitoramento e avaliação dos resultados de aprendizagem e replanejamento
    *   Monitoramento da implementação das ações recomposição das aprendizagens e avaliação dos resultados

In [42]:
# === IMPORTAÇÕES ===
import re
import pandas as pd
from pathlib import Path

# === FUNÇÕES ===

def is_devolutiva_file(filename):
    return "Devolutiva" in filename

def get_dimensao_subdimensao_from_filename(filename):
    partes = filename.stem.split("_")
    if len(partes) >= 5:
        return partes[3], "_".join(partes[4:])
    return "desconhecida", "desconhecida"

def extrair_blocos_devolutiva(markdown_text):
    blocos = re.split(r'(?=^### Rubrica \d+ – .+? – Nível .+$)', markdown_text, flags=re.MULTILINE)
    registros = []

    for bloco in blocos:
        bloco = bloco.strip()
        if not bloco.startswith("### Rubrica"):
            continue

        cabecalho = re.match(r'^### Rubrica (\d+) – (.+?) – Nível (.+)$', bloco, flags=re.MULTILINE)
        if not cabecalho:
            continue

        rubrica_num = cabecalho.group(1).strip()
        rubrica_nome = cabecalho.group(2).strip()
        nivel = cabecalho.group(3).strip()

        conteudo = bloco[cabecalho.end():].strip()

        secao_regex = re.compile(
            r'\*\*Seus pontos fortes\*\*[:]*\s*(.*?)\s*'
            r'\*\*O que fazer para avançar\?\*\*\s*(.*?)\s*'
            r'\*\*Necessidades formativas\*\*\s*(.*)',
            flags=re.DOTALL
        )

        secoes = secao_regex.search(conteudo)
        if not secoes:
            continue

        pontos_fortes = secoes.group(1).strip()
        avancar = secoes.group(2).strip()
        formativas = secoes.group(3).strip()

        registros.append({
            "rubrica": f"Rubrica {rubrica_num} - {rubrica_nome}",
            "nivel": nivel,
            "pontos_fortes": pontos_fortes,
            "avancar": avancar,
            "formativas": formativas
        })
    return registros

def processar_devolutivas_em_pasta(pasta_markdowns: Path):
    arquivos_md = list(pasta_markdowns.rglob("*.md"))
    todos_registros = []

    for arquivo in arquivos_md:
        if not is_devolutiva_file(arquivo.name):
            continue

        with open(arquivo, encoding="utf-8") as f:
            texto = f.read()

        registros = extrair_blocos_devolutiva(texto)
        dimensao, subdimensao = get_dimensao_subdimensao_from_filename(arquivo)

        for r in registros:
            r["dimensao"] = dimensao
            r["subdimensao"] = subdimensao
            r["arquivo_origem"] = arquivo.name
            todos_registros.append(r)

    return pd.DataFrame(todos_registros)

def pontuacao_para_rubrica_nivel(pontuacao: int):
    if 0 <= pontuacao <= 5:
        return "Rubrica 1 - Sensibilização", "Consolidar"
    elif 6 <= pontuacao <= 9:
        return "Rubrica 1 - Sensibilização", "Avançar"
    elif 10 <= pontuacao <= 16:
        return "Rubrica 2 - Exploração", "Consolidar"
    elif 17 <= pontuacao <= 21:
        return "Rubrica 2 - Exploração", "Avançar"
    elif 22 <= pontuacao <= 30:
        return "Rubrica 3 - Liderança Estratégica", "Consolidar"
    elif 31 <= pontuacao <= 36:
        return "Rubrica 3 - Liderança Estratégica", "Avançar"
    elif 37 <= pontuacao <= 51:
        return "Rubrica 4 - Transformação cultural", "Consolidar"
    else:
        return None, None

def gerar_devolutiva_dimensao(pontuacao: int, dimensao: str, subdimensao: str, df: pd.DataFrame) -> str:
    rubrica, nivel = pontuacao_para_rubrica_nivel(pontuacao)
    if not rubrica:
        return f"❌ Pontuação {pontuacao} está fora da faixa válida (0 a 51)."

    df_filtrado = df[
        (df["dimensao"] == dimensao) &
        (df["subdimensao"] == subdimensao)
    ]

    resultado = df_filtrado[
        (df_filtrado["rubrica"] == rubrica) &
        (df_filtrado["nivel"] == nivel)
    ]

    if resultado.empty:
        return f"❌ Não foi possível encontrar a devolutiva para {rubrica} - {nivel} na subdimensão '{subdimensao}'."

    item = resultado.iloc[0]
    texto = f"""
🔹 **Pontuação:** {pontuacao}
📂 **Dimensão:** {dimensao}
📁 **Subdimensão:** {subdimensao}
🏷️ **Rubrica:** {item['rubrica']}
📊 **Nível:** {item['nivel']}

---

✅ **Seus pontos fortes:**

{item['pontos_fortes']}

---

📈 **O que fazer para avançar:**

{item['avancar']}

---

📚 **Necessidades formativas:**

{item['formativas']}
"""
    return texto.strip()

In [44]:
# === USO ===
# Exemplo de uso após extrair os arquivos markdown
pasta_markdowns = Path("markdowns")
df_todas_devolutivas = processar_devolutivas_em_pasta(pasta_markdowns)
texto = gerar_devolutiva_dimensao(17, "pedag", "planejamento", df_todas_devolutivas)
Markdown(texto)

🔹 **Pontuação:** 17
📂 **Dimensão:** pedag
📁 **Subdimensão:** planejamento
🏷️ **Rubrica:** Rubrica 2 - Exploração
📊 **Nível:** Avançar

---

✅ **Seus pontos fortes:**

Parabéns! Você já se sente preparado(a) para desempenhar um papel ativo na mediação do planejamento pedagógico, garantindo que ele seja baseado em um diagnóstico de aprendizagem dos estudantes. Você se percebe preparado(a) para promover discussões importantes sobre o alinhamento do planejamento com as políticas educacionais e a proposta pedagógica da escola, além de monitorar se as ações estão sendo implementadas conforme planejado.

---

📈 **O que fazer para avançar:**

Para avançar, será fundamental estar preparado(a) para ampliar o escopo do diagnóstico, estruturar melhor as rotinas de monitoramento e replanejamento e fornecer apoio consistente aos professores, com diretrizes claras sobre o que deve conter no planejamento e devolutivas construtivas. Isso inclui estar preparado para continuamente avaliar e revisar as estratégias de recomposição das aprendizagens. Veja a seguir algumas indicações de objetos educacionais e recursos de aprendizagem que podem apoiá-lo nessa tarefa.

---

📚 **Necessidades formativas:**

1.  **Diagnóstico escolar para embasar o planejamento pedagógico**
    *   Diagnóstico das práticas de ensino dos professores (sua capacidade de transpor os parâmetros curriculares em práticas efetivas de ensino e aprendizagem)
2.  **Planejamento das ações, responsáveis e recursos necessários**
    *   Conhecimento de estratégias didático-metodológicas para aprendizagem mais efetiva (metodologias ativas, aprendizagem por projetos, práticas de diferenciação e personalização do ensino)
    *   Elaboração de diretrizes para o planejamento macro das ações e para o seu desdobramento progressivo em planos de aula detalhados
    *   Conhecimento de estratégias específicas de recomposição das aprendizagens e diretrizes para sua integração ao planejamento pedagógico.
3.  **Suporte e devolutiva aos professores**
    *   Avaliação e devolutiva técnico/crítica do planejamento pedagógico dos professores (critérios de qualidade, roteiros de análise e devolutiva, técnicas de feedback do gestor e entre pares).
4.  **Monitoramento, avaliação e replanejamento**
    *   Monitoramento e avaliação dos resultados de aprendizagem e replanejamento
    *   Monitoramento da implementação das ações recomposição das aprendizagens e avaliação dos resultados